# Establishing a Good Distance Between Flu Sequences

First we need to establish the meaning of a distance between Influenza sequences. We will look at a special subset of the data where we know roughly what the similarity is bewtween a pair. 

## Jaccard with k-grams

We will run k-grams and Jaccard similarity on the sequences and cross validate and see if the ones who are similar match our domain knowledge.

In [1]:
import pickle
import gzip
import cProfile
from FASTA_parse import sequence

In [2]:
# read in the sequences
with gzip.open('influenza.fna.pklz', 'rb') as fin:
    fna = pickle.load(fin)

In [3]:
fna_meta, fna_seq = fna.get_dataframe()

In [6]:
fna_seq.describe()

,Sequence
count,655763
unique,441220
top,ATGAAGACAACCATTATTTTGATACTACTGACCCATTGGGTCTACA...
freq,2481


In [7]:
fna_meta.describe()

,NCBI_1,Key_1,NCBI_2,Key_2,Details
count,655764,655764,655764,655764,655764
unique,1,655764,1,655764,629491
top,gi,295915982,gb,CY090021,Influenza A virus (A/Wisconsin/67/2005(H3N2)) ...
freq,655764,1,655764,1,45
